In [ ]:
import torch
from torch.autograd import Variable
from model import UNet
import os
import numpy as np
os.environ['CUDA_VISIBLE_DEVICES']='0'

### Model forward

In [ ]:
input_file = '/mnt/sdb1/intern_data/pix2pix_wbce_pet3d/train/1_input.npy'
input_arr = np.load(input_file).reshape(1,1,116,132,132)
input_arr = Variable(torch.from_numpy(input_arr).float().cuda(), volatile = False) # set volatile to false for training
model = UNet(n_ch = 1, n_class = 1).cuda()
pred_arr = model(input_arr)
pred_arr = pred_arr.data.cpu().numpy()
pred_arr = pred_arr.reshape(28,44,44)
print(pred_arr.shape)

### Get corresponding ground truths

In [ ]:
target_file = '/mnt/sdb1/intern_data/pix2pix_wbce_pet3d/train/1_target.npy'
target_arr = np.load(target_file)
shape = target_arr.shape
print('Raw target shape : ', target_arr.shape)
target_arr = target_arr[shape[0]//2-14:shape[0]//2+14, shape[1]//2-22:shape[1]//2+22, shape[2]//2-22:shape[2]//2+22]
print('Cropped target shape : ', target_arr.shape) # this is the image to train the model with

### Defining Loss

In [ ]:
def bce_loss(scores, target):
    neg_abs = -scores.abs()
    loss = scores.clamp(min=0) - scores * target + (1 + neg_abs.exp()).log()
    return loss.mean()

In [ ]:
def generator_loss(fake_logits):
    loss = bce_loss(fake_logits, 1)
    return loss

In [ ]:
def discriminator_loss(real_logits, fake_logits):
    loss = bce_loss(real_logits, 1) + bce_loss(fake_logits, 0)
    return loss

### Defining Optimizer

In [ ]:
def get_optimizer(model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, betas=(0.5, 0.999))
    return optimizer

### Defining train function

In [ ]:
def train(Discriminator, Generator, D_Opti, G_Opti, D_Loss, G_Loss, print_every=100, batch_size=32, num_epochs=20):
    i = 0
    for epoch in range(num_epochs):
        for _ in range(train_size // batch_size):
            input, target = data_batcher.getNextBatch(batch_size)
            
            D_Opti.zero_grad()
            input_data = Variable(input).type(dtype)
            target_data = Variable(target).type(dtype)
            logits_real = Discriminator(target_data)
            fake_images = Generator(input_data)
            logits_fake = Discriminator(fake_images)
            DiscLoss = D_Loss(logits_real, logits_fake)
            DiscLoss.backward()
            D_Opti.step()
            
            G_opti.zero_grad()
            GenLoss = G_Loss(logits_fake)
            GenLoss.backward()
            G_Opti.step()
            
            if(i % print_every == 0):
                print("Discriminator Loss = ", DiscLoss)
                print("Generator Loss = ", GenLoss)
                
            i+= 1